In [46]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [48]:
#Reading user file:
user_cols =  ['user_id', 'products', 'reviews']
users = pd.read_csv('users.csv', nrows=1000 , sep=',', names=user_cols, encoding='latin-1')

n_users = users.shape[0]
print('Number of users:', n_users)

Number of users: 1000


In [62]:
#Reading games file:

games_cols = ['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating', 'positive_ratio', 'user_reviews', 'price_final', 'price_original', 'discount', 'steam_deck']
games = pd.read_csv('games.csv', nrows=1000, sep=',', names=games_cols, encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('games.csv', nrows=1000, sep='\t', names=r_cols, encoding='latin-1')

rate_games = ratings_base.values

print ('Number of rates:', rate_games.shape[0])


Number of rates: 1000


In [60]:
#Reading recommendations file:
games_metadata_cols = ['app_id', 'description', 'tags']

games_metadata = pd.read_csv('games_metadata.csv', nrows=1000, sep='\t', names=games_metadata_cols, encoding='latin-1')

# Remove rows with NaN values
games_metadata = games_metadata.dropna()

games_metadata_train = games_metadata.values
games_metadata_test = games_metadata.values

print('Number of training rates:', games_metadata_train)

Number of training rates: []


In [49]:
#Reading recommendations file:
recommendations_cols = ['app_id', 'helpful', 'funny', 'date', 'is_recommended', 'hours', 'user_id', 'review_id']

recommendations = pd.read_csv('recommendations.csv', nrows=1000, sep='\t', names=recommendations_cols, encoding='latin-1')

recommendations_train = recommendations.shape[0]
#recommendations_test = recommendations.as_matrix()

print ('Number of traing rates:', recommendations_train)
#print ('Number of test rates:', recommendations_test.shape[0])

Number of traing rates: 1000


In [51]:
#X_train_counts = games['rating'].values
X0 = games.values
X_train_counts = X0[:, 8:11]

In [52]:
#from sklearn.feature_extraction.text import TfidfTransformer
#transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
#tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer

# Convert X_train_counts to a 2D array
X_train_counts_2d = FunctionTransformer(lambda x: x.reshape(-1, 1)).fit_transform(X_train_counts)

# Create and apply the TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm='l2')
tfidf = transformer.fit_transform(X_train_counts_2d).toarray()

In [53]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores), time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # while index in python starts from 0
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0 
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

In [57]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
d = tfidf.shape[1]  # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):
    ids, scores = get_items_rated_by_user(recommendations, n)  # Assuming `recommendations` is the correct variable
    clf = Ridge(alpha=0.01, fit_intercept=True)
    Xhat = tfidf[ids, :]
    
    clf.fit(Xhat, scores)
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

InvalidIndexError: (slice(None, None, None), 0)

In [55]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):    
    ids, scores = get_items_rated_by_user(recommendations_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]
    
    clf.fit(Xhat, scores) 
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

TypeError: 'int' object is not subscriptable